In [1]:
import os
import sys
os.chdir('../')

In [2]:
pwd

'c:\\Users\\Viral Sherathiya\\Desktop\\NLP Data\\Text_Summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    metric_file_name: Path
    tokenizer_path: Path

In [4]:
from textSummarization.constants import *
from textSummarization.utils.common import *
from textSummarization import logger, CustomException


class ConfigurationManager:

    def __init__(
            self,
            config_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        try:
            config = self.config.model_trainer
            create_directories([config.root_dir])

            model_evaluation_config = ModelEvaluationConfig(
                root_dir=config.root_dir,
                data_path=config.data_path,
                model_path=config.model_path,
                metric_file_name=config.metric_file_name,
                tokenizer_path=config.tokenizer_path
            )

            return model_evaluation_config
        except Exception as e:
            logger.info(f"Error Occurred at {CustomException(e,sys)}")
            raise CustomException(e, sys)

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk, load_metric
import torch
from tqdm import tqdm
import pandas as pd

# class ModelEvaluation:
#     def __init__(self,config:ModelEvaluationConfig):
#         self.config = config


#     def generate_batch_chunks(self,list_of_elements,batch_size):
#         try:
#             for i in range(0,len(list_of_elements),batch_size):
#                 yield list_of_elements[i:i+batch_size]
#         except Exception as e:
#                 logger.info(f"Error Occurred at {CustomException(e,sys)}")
#                 raise CustomException(e, sys)


#     def calculate_metric_on_test_ds(self,dataset,metric,model,tokenizer,
#                                     device="cuda" if torch.cuda.is_available() else "cpu",
#                                     batch_size=16,
#                                     column_text ="article",column_summary = "highlights",
#                                     ):
#         try:
#             article_batches = list(self.generate_batch_chunks(dataset[column_text],batch_size=batch_size))
#             target_batches = list(self.generate_batch_chunks(dataset[column_summary],batch_size=batch_size))


#             for article_batch,target_batch in tqdm(
#                 zip(article_batches,target_batches),total=len(article_batches)):

#                 inputs = tokenizer(article_batch,max_length =1024,truncation =True,
#                                 padding = 'max_length',return_tensor = 'pt')

#                 summaries = model.generate(inputs_ids = inputs['input_ids'].to(device),
#                                         attention_mask = inputs['attention_mask'].to(device),
#                                         max_length = 128,
#                                         num_beams = 8,
#                                         length_penalty = 0.8)

#                 ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

#                 # Finally, we decode the generated texts,
#                 # replace the  token, and add the decoded texts with the references to the metric.
#                 decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
#                                         clean_up_tokenization_spaces=True)
#                     for s in summaries]

#                 decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


#                 metric.add_batch(predictions=decoded_summaries, references=target_batch)

#             score = metric.compute()
#             return score
#         except Exception as e:
#                 logger.info(f"Error Occurred at {CustomException(e,sys)}")
#                 raise CustomException(e, sys)


#     def evaluate(self):
#         try:
#             device = "cuda" if torch.cuda.is_available() else "cpu"
#             tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
#             model_pegasus =  AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)


#             dataset_samsum_pt = load_from_disk(self.config.data_path)

#             rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

#             rouge_metric = load_metric('rouge')

#             score = self.calculate_metric_on_test_ds(
#             dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
#                 )

#             rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

#             df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
#             df.to_csv(self.config.metric_file_name, index=False)


#         except Exception as e:
#                 logger.info(f"Error Occurred at {CustomException(e,sys)}")
#                 raise CustomException(e, sys)

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i: i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                    batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                                    column_text="article",
                                    column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(
            dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(
            dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
                zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                               padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                       attention_mask=inputs["attention_mask"].to(
                                           device),
                                       length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

            # Finally, we decode the generated texts,
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                                  clean_up_tokenization_spaces=True)
                                 for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries,
                             references=target_batch)

        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(
            self.config.model_path).to(device)

        # loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size=2, column_text='dialogue', column_summary='summary'
        )

        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    logger.info(f"Error Occurred at {CustomException(e,sys)}")
    raise CustomException(e, sys)

[2023-08-13 23:12:14,084] || common.py || 37 || TextSummarizerLogger || read_yaml() || 37 || INFO || yaml file : config\config.yaml loaded successfully
[2023-08-13 23:12:14,086] || common.py || 37 || TextSummarizerLogger || read_yaml() || 37 || INFO || yaml file : config\params.yaml loaded successfully
[2023-08-13 23:12:14,087] || common.py || 69 || TextSummarizerLogger || create_directories() || 69 || INFO || Directory artifacts created successfully
[2023-08-13 23:12:14,088] || common.py || 69 || TextSummarizerLogger || create_directories() || 69 || INFO || Directory artifacts/model_trainer created successfully
[2023-08-13 23:12:14,089] || 4019877226.py || 34 || TextSummarizerLogger || get_model_evaluation_config() || 34 || INFO || Error Occurred at Python Script :
    [C:\Users\Viral Sherathiya\AppData\Local\Temp\ipykernel_14236\4019877226.py] 
    at try block line number : [27] and exception block line no : [34] 
    error message : 
    ["'ConfigBox' object has no attribute 'model

CustomException: Python Script :
    [C:\Users\Viral Sherathiya\AppData\Local\Temp\ipykernel_14236\922687971.py] 
    at try block line number : [3] and exception block line no : [8] 
    error message : 
    [Python Script :
    [C:\Users\Viral Sherathiya\AppData\Local\Temp\ipykernel_14236\4019877226.py] 
    at try block line number : [27] and exception block line no : [35] 
    error message : 
    ["'ConfigBox' object has no attribute 'model_path'"]
    ]
    